# 机器学习基石 - hw1 - Q18~20

## 一、训练

### 1.1 从文件读取训练集

In [1]:
import pandas as pd
import numpy as np

# 读取 .dat 文件，names 指定列名，delim_whitespace 设为 True 等同于设置 sep='\s+'
dataset_train = pd.read_csv("resource/hw1_18_train.dat", names=['x1', 'x2', 'x3', 'x4', 'y'], delim_whitespace=True)

# 查看前指定行数据（默认为 5），检查是否正确读取 / dataset.tail(n) 查看末几行
dataset_train.head()

,x1,x2,x3,x4,y
0,0.94544,0.428420,0.798330,0.162440,-1
1,0.85365,0.084168,0.568200,0.492210,-1
2,0.17095,0.821270,0.984440,0.514860,-1
3,0.51412,0.921240,0.423230,0.097934,-1
4,0.28147,0.714340,0.075309,0.911600,1


In [2]:
# 查看 technical information（其中 entries 指“条目”，即有多少行）
dataset_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
x1    500 non-null float64
x2    500 non-null float64
x3    500 non-null float64
x4    500 non-null float64
y     500 non-null int64
dtypes: float64(4), int64(1)
memory usage: 19.7 KB


### 1.2 数据集插入列

**将 x0 插入到数据表中**：[`DataFrame.insert`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.insert.html)

In [3]:
dataset_train.insert(0, 'x0', 1)  # 注意 x0 = 1 而不是 0 ……

dataset_train

,x0,x1,x2,x3,x4,y
0,1,0.945440,0.428420,0.798330,0.162440,-1
1,1,0.853650,0.084168,0.568200,0.492210,-1
2,1,0.170950,0.821270,0.984440,0.514860,-1
3,1,0.514120,0.921240,0.423230,0.097934,-1
4,1,0.281470,0.714340,0.075309,0.911600,1
...,...,...,...,...,...,...
495,1,0.542960,0.959950,0.384460,0.111450,-1
496,1,0.374380,0.128740,0.752800,0.977680,1
497,1,0.078827,0.382300,0.139650,0.826770,1
498,1,0.922880,0.863130,0.229390,0.922140,-1


### 1.3 用矩阵存储自变量与因变量

In [4]:
# 第一个 ':' 表示提取数据集的全部行，':-1' 表示提取除最后一列外的所有列
X_train = dataset_train.iloc[:, :-1].values

X_train

array([[1.      , 0.94544 , 0.42842 , 0.79833 , 0.16244 ],
       [1.      , 0.85365 , 0.084168, 0.5682  , 0.49221 ],
       [1.      , 0.17095 , 0.82127 , 0.98444 , 0.51486 ],
       ...,
       [1.      , 0.078827, 0.3823  , 0.13965 , 0.82677 ],
       [1.      , 0.92288 , 0.86313 , 0.22939 , 0.92214 ],
       [1.      , 0.50315 , 0.41884 , 0.026094, 0.91623 ]])

In [5]:
y_train = dataset_train.iloc[:, -1].values
y_train

array([-1, -1, -1, -1,  1, -1, -1, -1, -1,  1, -1, -1,  1, -1, -1,  1,  1,
       -1, -1,  1,  1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1,  1,  1, -1,  1,  1,  1, -1, -1, -1, -1,
        1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1, -1, -1,  1,
        1, -1,  1,  1,  1, -1,  1,  1,  1,  1, -1, -1,  1,  1, -1,  1, -1,
       -1, -1, -1, -1, -1, -1,  1, -1,  1,  1, -1, -1, -1, -1, -1,  1, -1,
        1, -1, -1,  1, -1,  1,  1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1,
       -1,  1,  1, -1,  1,  1,  1, -1, -1, -1,  1,  1,  1, -1, -1, -1, -1,
        1, -1,  1,  1,  1, -1, -1, -1, -1,  1, -1, -1,  1,  1,  1,  1, -1,
       -1,  1, -1, -1, -1,  1,  1, -1,  1,  1, -1,  1,  1,  1, -1,  1, -1,
        1, -1, -1, -1, -1,  1,  1,  1, -1,  1, -1,  1, -1,  1,  1,  1,  1,
       -1,  1, -1, -1,  1, -1,  1,  1, -1, -1,  1,  1, -1,  1, -1, -1, -1,
       -1,  1,  1,  1,  1, -1, -1, -1, -1,  1,  1, -1, -1, -1,  1, -1, -1,
        1, -1,  1, -1, -1

[numpy](https://numpy.org/devdocs/user/quickstart.html#the-basics) 最重要的数据类型就是 `ndarray`，即多维矩阵

In [6]:
# 查看 ndarray 的形状
X_train.shape

(500, 5)

## 三、Pocket PLA 算法

### 3.1 定义 sign 函数

In [7]:
def sign(n):
    if n > 0:
        return 1
    else:
        return -1

### 3.2 定义 w 更新规则

In [8]:
def pla_update(wt, xt, yt):
    """根据规则更新 w"""
    w_updated = wt + yt * xt
    return w_updated

In [9]:
def is_error(wt, xt, yt):
    """判断当前点是否正确"""
    if sign(np.vdot(wt, xt)) != yt:  # 错误点
        return True
    else:
        return False

In [10]:
def count_error(wt, X, y):
    """计算当前 w 在指定数据集中的总错误数"""
    cnt = 0
    for i in range(len(X)):        
        if is_error(wt, X[i], y[i]):
            cnt += 1
    
    return cnt

print(count_error(np.zeros(5), X_train, y_train))

195


### 3.3 定义 Pocket 执行过程

题目要求随机访问所有点

* ndarray 如果直接用 `arr2 = arr1` 拷贝，则 `arr2 is arr1` 为 `True`，改变其中任一个，也会造成另一个的改变
* 为了仅拷贝值，使其是两个不同的对象，不会关联改变，需要用 `arr2 = arr1.copy()`

一开始没看懂题目中指的 50 次 updates 是指找到 50 个错误点，还是指 pocket 中的 w 更新了 50 次。实验时发现，如果按照 pocket 中 w 更新 50 次的意思，则程序无法运行出结果，所以采用找到 50 次错误点进行 updates 为题意。

看了第 19 题发现确实如上分析所说，50 次 updates 是指 50 次 PLA updates.

In [11]:
def pocket_train(w0, X, y, update_func, t):
    """
    
    t: 修正 t 次后停止
    """
    wp = w0.copy()
    err_cnt_wp = count_error(wp, X, y)
    wt = w0.copy()
    ti = 0  # 当前已修正次数
    
    while ti < t:
        i = np.random.randint(0, len(X))
        if is_error(wt, X[i], y[i]):
            wt = update_func(wt, X[i], y[i])
            err_cnt_wt = count_error(wt, X, y)
            ti += 1
            # print(f"{ti} updates: {err_cnt_wt} erros")
            if err_cnt_wt < err_cnt_wp:  # TODO: 优化 - 数到比 err_cnt_wp 多就不数了，直接放弃
                wp = wt.copy()
                err_cnt_wp = err_cnt_wt
                # print(f"wp = {wp}")
                
    return wp, wt


T_UPDATES = 50
# 初始化 w0
w0 = np.zeros(5)
wp, wt = pocket_train(w0, X_train, y_train, pla_update, T_UPDATES)
print("------------------------------------------------------")
print(f"wp after {T_UPDATES} updates:", wp)
print(f"wt after {T_UPDATES} updates:", wt)
print("------------------------------------------------------")

------------------------------------------------------
wp after 50 updates: [ 1.       -1.437862 -1.813246 -1.853136  1.784002]
wt after 50 updates: [ 2.       -0.947517 -1.623476 -1.178494  2.072251]
------------------------------------------------------


## 二、测试

### 2.1 从文件读取测试数据

In [12]:
dataset_test = pd.read_csv("resource/hw1_18_test.dat", names=['x1', 'x2', 'x3', 'x4', 'y'], delim_whitespace=True)
dataset_test.head()

,x1,x2,x3,x4,y
0,0.629260,0.32783,0.010417,0.731020,1
1,0.323680,0.61439,0.420970,0.025626,-1
2,0.159680,0.83346,0.975150,0.327620,-1
3,0.023526,0.30292,0.014961,0.922880,1
4,0.458040,0.74520,0.834060,0.549320,1


In [13]:
dataset_test.insert(0, 'x0', 1)
dataset_test.head()

,x0,x1,x2,x3,x4,y
0,1,0.629260,0.32783,0.010417,0.731020,1
1,1,0.323680,0.61439,0.420970,0.025626,-1
2,1,0.159680,0.83346,0.975150,0.327620,-1
3,1,0.023526,0.30292,0.014961,0.922880,1
4,1,0.458040,0.74520,0.834060,0.549320,1


In [14]:
X_test = dataset_test.iloc[:, :-1].values
y_test = dataset_test.iloc[:, -1].values

print("X_test:")
print(X_test[:5])  # 查看前五行
print("y_test:")
print(y_test[:5])

X_test:
[[1.       0.62926  0.32783  0.010417 0.73102 ]
 [1.       0.32368  0.61439  0.42097  0.025626]
 [1.       0.15968  0.83346  0.97515  0.32762 ]
 [1.       0.023526 0.30292  0.014961 0.92288 ]
 [1.       0.45804  0.7452   0.83406  0.54932 ]]
y_test:
[ 1 -1 -1  1  1]


In [15]:
def pocket_test(wp, X, y, update_func):
    """
    wp: w in the pocket
    """
    err_cnt = count_error(wp, X, y)
    err_rate = err_cnt / len(X)
    return err_rate

err_rate = pocket_test(wp, X_test, y_test, pla_update)
print("------------------------------------------------------")
print(f"wp {wp}: error rate = {err_rate}")
print("------------------------------------------------------")


------------------------------------------------------
wp [ 1.       -1.437862 -1.813246 -1.853136  1.784002]: error rate = 0.114
------------------------------------------------------


## Q18 - Q19

In [16]:
T_UPDATES = 50
T_REPEAT = 2000
w0 = np.zeros(5)

sum_err_rate_wp = 0
sum_err_rate_wt = 0
for i in range(T_REPEAT):
    # train
    wp, wt = pocket_train(w0, X_train, y_train, pla_update, T_UPDATES)
    # test - wp
    err_rate_wp = pocket_test(wp, X_test, y_test, pla_update)
    sum_err_rate_wp += err_rate_wp
    # test - wt
    err_rate_wt = pocket_test(wt, X_test, y_test, pla_update)
    sum_err_rate_wt += err_rate_wt
    print(f"exp{i} - wp {wp}: error rate = {err_rate_wp}")
    print(f"exp{i} - wt {wt}: error rate = {err_rate_wt}")
    
avg_err_rate_wp = sum_err_rate_wp / T_REPEAT
avg_err_rate_wt = sum_err_rate_wt / T_REPEAT
print("------------------------------------------------------")
print(f"Average wp Error Rate in {T_REPEAT} Experiments: {avg_err_rate_wp}")
print(f"Average wt Error Rate in {T_REPEAT} Experiments: {avg_err_rate_wt}")
print("------------------------------------------------------")


exp0 - wp [ 1.        -2.375248  -1.4221292 -1.277919   1.64449  ]: error rate = 0.13
exp0 - wt [ 2.        -1.790328  -1.3365122 -0.644099   2.39486  ]: error rate = 0.618
exp1 - wp [ 2.        -2.541525  -2.5323853 -1.688454   1.762132 ]: error rate = 0.128
exp1 - wt [ 2.        -2.998005  -2.5474453 -1.098134   2.308872 ]: error rate = 0.196
exp2 - wp [ 0.       -1.087147 -1.98044  -1.722354  3.082612]: error rate = 0.142
exp2 - wt [ 0.       -3.220571 -2.754759 -0.273493  1.318577]: error rate = 0.29
exp3 - wp [ 1.         -0.862212   -1.8290245  -0.73449     1.08271008]: error rate = 0.188
exp3 - wt [ 0.         -2.283632   -1.6659145  -1.058312    1.06257008]: error rate = 0.29
exp4 - wp [ 0.       -0.867195 -1.200409 -0.949733  1.927663]: error rate = 0.134
exp4 - wt [ 0.       -0.781963 -2.861643 -0.972137  2.833048]: error rate = 0.166
exp5 - wp [ 1.        -1.536531  -3.09486   -1.7837707  3.065927 ]: error rate = 0.112
exp5 - wt [ 0.        -2.147731  -3.125628  -2.2197007  

exp50 - wp [ 1.        -1.659272  -2.6140395 -0.992642   2.4660677]: error rate = 0.126
exp50 - wt [ 0.        -1.314225  -1.8044275  0.255242   2.5313577]: error rate = 0.22
exp51 - wp [ 1.         -1.596891   -2.47034061 -1.624961    2.343778  ]: error rate = 0.1
exp51 - wt [ 2.         -0.576171   -2.44471061 -0.690639    2.147568  ]: error rate = 0.598
exp52 - wp [ 1.       -0.77327  -1.486742 -1.045425  0.96437 ]: error rate = 0.176
exp52 - wt [ 2.       -1.577964 -3.662771 -1.301255  3.245362]: error rate = 0.312
exp53 - wp [ 1.       -1.487908 -1.912264 -1.292266  1.69331 ]: error rate = 0.1
exp53 - wt [ 0.       -2.318161 -2.558905 -1.852434  1.22531 ]: error rate = 0.292
exp54 - wp [ 1.        -2.103327  -1.5004985 -1.6768586  1.9333627]: error rate = 0.12
exp54 - wt [ 0.        -1.656704  -2.6524885 -0.5249196  2.1472627]: error rate = 0.206
exp55 - wp [ 2.        -2.388416  -2.5021455 -1.413873   1.5446427]: error rate = 0.15
exp55 - wt [ 2.        -2.674226  -2.3024955 -1.8

exp98 - wp [ 2.        -3.205196  -2.5864554 -1.747851   2.58631  ]: error rate = 0.134
exp98 - wt [ 2.        -3.205196  -2.5864554 -1.747851   2.58631  ]: error rate = 0.134
exp99 - wp [ 1.       -1.955278 -1.867125 -0.98067   1.755875]: error rate = 0.104
exp99 - wt [ 0.        -2.845725  -1.8136935 -1.9242353  1.9263477]: error rate = 0.274
exp100 - wp [ 1.       -1.39454  -1.550002 -0.955769  1.00056 ]: error rate = 0.118
exp100 - wt [ 2.       -2.156208 -1.361418 -1.235094  2.207053]: error rate = 0.448
exp101 - wp [ 1.       -1.482123 -1.959125 -0.430893  1.390569]: error rate = 0.158
exp101 - wt [ 2.        -1.760783  -3.9814317 -2.335835   0.8942817]: error rate = 0.18
exp102 - wp [ 1.       -0.935997 -2.278158 -0.783     1.448544]: error rate = 0.172
exp102 - wt [ 2.       -1.246242 -1.340308 -2.275349  2.489997]: error rate = 0.458
exp103 - wp [ 0.       -1.11644  -1.41901  -0.778053  2.341163]: error rate = 0.136
exp103 - wt [ 2.         -2.825153   -1.27168561 -0.381694   

exp145 - wt [ 2.         -1.6238     -2.616101   -0.557464    4.01028838]: error rate = 0.55
exp146 - wp [ 1.       -1.658925 -2.558357 -1.311441  2.183465]: error rate = 0.102
exp146 - wt [ 0.       -1.887144 -3.96791  -2.135236  2.575869]: error rate = 0.262
exp147 - wp [ 1.       -2.114123 -1.473182 -1.861564  2.370524]: error rate = 0.128
exp147 - wt [ 0.       -2.578283 -1.416292 -1.735144  2.474474]: error rate = 0.224
exp148 - wp [ 1.         -2.744293   -2.34931911 -1.461296    2.8722767 ]: error rate = 0.108
exp148 - wt [ 2.         -2.493583   -3.40393611 -0.307316    2.8976321 ]: error rate = 0.304
exp149 - wp [ 1.       -1.21932  -1.487858 -0.922951  0.995648]: error rate = 0.114
exp149 - wt [ 0.        -2.418501  -1.7745157 -1.107379   2.2365057]: error rate = 0.224
exp150 - wp [ 1.        -2.163387  -2.348396  -1.558159   3.1738614]: error rate = 0.122
exp150 - wt [ 2.        -2.09235   -1.406176  -0.905659   3.2647164]: error rate = 0.57
exp151 - wp [ 1.        -1.377336

exp194 - wp [ 1.        -1.816277  -2.5247561 -0.858783   2.17177  ]: error rate = 0.11
exp194 - wt [ 2.        -2.589629  -2.7168961 -1.806954   3.213865 ]: error rate = 0.244
exp195 - wp [ 1.       -1.002172 -1.539559 -0.508402  0.77733 ]: error rate = 0.188
exp195 - wt [ 2.        -2.995639  -1.9656655 -1.127189  -0.3767073]: error rate = 0.224
exp196 - wp [ 1.         -1.474814   -1.77413551 -1.41143     1.64054   ]: error rate = 0.102
exp196 - wt [ 0.         -0.850205   -2.44653551 -1.199964    1.56478   ]: error rate = 0.244
exp197 - wp [ 1.       -2.343592 -2.734851 -0.923449  2.73041 ]: error rate = 0.114
exp197 - wt [ 2.       -1.890748 -2.947565 -1.042549  3.963099]: error rate = 0.44
exp198 - wp [ 0.        -1.808288  -2.1990062 -0.705252   2.743688 ]: error rate = 0.152
exp198 - wt [ 0.        -3.332848  -1.6299332 -1.85858    1.765208 ]: error rate = 0.282
exp199 - wp [ 2.       -1.713838 -2.949911 -1.370366  1.148982]: error rate = 0.168
exp199 - wt [ 2.        -3.494954

...

exp1787 - wp [ 0.        -1.782641  -1.40402   -0.9073375  2.786405 ]: error rate = 0.142
exp1787 - wt [ 0.         -2.700225   -3.18365391 -2.1953225   3.797941  ]: error rate = 0.206
exp1788 - wp [ 1.       -1.277302 -2.868079 -1.244262  2.578296]: error rate = 0.144
exp1788 - wt [ 0.       -1.834242 -3.527706 -2.167363  2.030396]: error rate = 0.282
exp1789 - wp [ 0.       -2.402288 -2.584269 -0.270015  3.29931 ]: error rate = 0.15
exp1789 - wt [ 0.       -2.402288 -2.584269 -0.270015  3.29931 ]: error rate = 0.15
exp1790 - wp [ 1.        -1.520879  -2.5320962 -1.053763   2.609978 ]: error rate = 0.16
exp1790 - wt [ 0.        -2.370665  -1.9579262 -1.598097   3.567858 ]: error rate = 0.156
exp1791 - wp [ 1.        -1.926622  -2.444719  -0.5658055  2.225504 ]: error rate = 0.142
exp1791 - wt [ 2.        -2.084345  -1.180628  -1.4240341  1.973346 ]: error rate = 0.434
exp1792 - wp [ 1.        -1.825935  -2.2499858 -2.649264   3.2224767]: error rate = 0.126
exp1792 - wt [ 0.        -1.

exp1835 - wp [ 2.        -2.414107  -2.4189465 -1.6983636  1.8572027]: error rate = 0.15
exp1835 - wt [ 2.        -2.643927  -2.3671887 -2.1946936  2.4852927]: error rate = 0.15
exp1836 - wp [ 1.       -1.111989 -2.073951 -1.331636  1.678737]: error rate = 0.114
exp1836 - wt [ 2.        0.182804 -1.006585 -1.568764  2.446132]: error rate = 0.698
exp1837 - wp [ 1.        -1.904072  -1.925035  -0.634331   1.6750951]: error rate = 0.126
exp1837 - wt [ 0.         -1.088272   -1.48127061 -1.306728    1.7825691 ]: error rate = 0.206
exp1838 - wp [ 0.       -0.936541 -1.879223 -1.593792  2.669415]: error rate = 0.156
exp1838 - wt [ 0.       -0.923328 -2.92392  -1.486008  2.236135]: error rate = 0.224
exp1839 - wp [ 1.       -1.786179 -2.304228 -1.544922  2.51319 ]: error rate = 0.1
exp1839 - wt [ 2.       -2.085041 -1.900435 -0.82584   3.197585]: error rate = 0.52
exp1840 - wp [ 1.       -1.896465 -1.60817  -1.060239  1.348266]: error rate = 0.118
exp1840 - wt [ 2.       -0.856522 -2.570001  

exp1882 - wp [ 0.        -0.957642  -2.1063423 -0.374674   2.6478713]: error rate = 0.154
exp1882 - wt [ 0.        -2.068093  -3.2203723 -0.777149   1.5007513]: error rate = 0.28
exp1883 - wp [ 0.        -1.805663  -1.6919669 -1.275494   3.506555 ]: error rate = 0.146
exp1883 - wt [ 2.        -2.941601  -0.9758729  0.630146   2.550775 ]: error rate = 0.626
exp1884 - wp [ 1.       -0.845922 -1.306805 -0.854115  0.929236]: error rate = 0.212
exp1884 - wt [ 0.       -1.212652 -1.515185 -1.303295  0.305636]: error rate = 0.298
exp1885 - wp [ 1.       -1.764312 -2.557837 -1.070833  1.919982]: error rate = 0.106
exp1885 - wt [ 2.       -1.934672 -2.755241 -0.371378  1.551277]: error rate = 0.294
exp1886 - wp [ 2.       -2.34796  -3.556484 -1.153921  1.94518 ]: error rate = 0.148
exp1886 - wt [ 2.       -2.46556  -2.689494 -1.536371  1.38508 ]: error rate = 0.13
exp1887 - wp [ 1.         -2.347247   -2.40848561 -0.898218    2.2574331 ]: error rate = 0.104
exp1887 - wt [ 2.         -1.468187  

exp1932 - wp [ 1.       -1.490697 -2.198528 -2.149337  2.299935]: error rate = 0.12
exp1932 - wt [ 2.       -1.212997 -1.808398 -1.210957  3.148995]: error rate = 0.572
exp1933 - wp [ 1.       -1.85027  -2.344539 -1.418637  2.234975]: error rate = 0.1
exp1933 - wt [ 2.         -1.240363   -2.93207561 -0.674732    2.40524   ]: error rate = 0.438
exp1934 - wp [ 2.        -2.204588  -3.4540335 -1.278746   2.4706767]: error rate = 0.196
exp1934 - wt [ 2.        -1.523278  -3.8612235 -1.289226   1.9196867]: error rate = 0.198
exp1935 - wp [ 1.        -1.38177   -2.1608288 -1.630048   2.2107717]: error rate = 0.106
exp1935 - wt [ 0.        -2.158639  -1.7795758 -1.518287   2.3864617]: error rate = 0.218
exp1936 - wp [ 1.       -1.70846  -2.072608 -0.735886  1.40489 ]: error rate = 0.11
exp1936 - wt [ 0.       -0.961121 -2.688055 -0.969626  0.145972]: error rate = 0.298
exp1937 - wp [ 1.        -1.586424  -1.3549363 -1.511155   1.98442  ]: error rate = 0.146
exp1937 - wt [ 0.        -2.797864

exp1981 - wp [ 0.        -0.814646  -1.315975  -1.360633   2.3587727]: error rate = 0.15
exp1981 - wt [ 0.        -0.639786  -3.031079  -0.135303   3.1974227]: error rate = 0.2
exp1982 - wp [ 1.        -0.658914  -1.5047869 -0.909597   0.83399  ]: error rate = 0.196
exp1982 - wt [ 2.        -0.978849  -2.3568189 -1.539528   3.383048 ]: error rate = 0.518
exp1983 - wp [ 1.        -2.841654  -1.7389263 -2.016245   2.963589 ]: error rate = 0.128
exp1983 - wt [ 2.        -2.014167  -2.3243011 -2.42046    2.1655587]: error rate = 0.184
exp1984 - wp [ 0.       -1.340097 -2.289663 -0.909067  2.687185]: error rate = 0.146
exp1984 - wt [ 0.       -1.36222  -2.646033 -1.241644  2.557855]: error rate = 0.19
exp1985 - wp [ 1.        -1.249494  -2.576208  -2.006516   2.4596541]: error rate = 0.126
exp1985 - wt [ 2.        -0.429674  -3.222068  -2.484936   1.9665891]: error rate = 0.254
exp1986 - wp [ 1.         -1.466917   -2.03562361 -1.253097    1.829569  ]: error rate = 0.1
exp1986 - wt [ 2.    

## Q20

In [17]:
T_UPDATES_Q20 = 100
T_REPEAT = 2000
w0 = np.zeros(5)

sum_err_rate_wp = 0
for i in range(T_REPEAT):
    # train
    wp, _ = pocket_train(w0, X_train, y_train, pla_update, T_UPDATES_Q20)
    # test - wp
    err_rate_wp = pocket_test(wp, X_test, y_test, pla_update)
    sum_err_rate_wp += err_rate_wp
    print(f"exp{i} - wp {wp}: error rate = {err_rate_wp}")
    
avg_err_rate_wp = sum_err_rate_wp / T_REPEAT
print("------------------------------------------------------")
print(f"Average wp Error Rate in {T_REPEAT} Experiments: {avg_err_rate_wp}")
print("------------------------------------------------------")

exp0 - wp [ 1.         -1.919172   -2.5619912  -2.070529    2.81347638]: error rate = 0.108
exp1 - wp [ 1.       -1.708413 -2.536539 -1.364768  2.387154]: error rate = 0.1
exp2 - wp [ 1.        -1.60688   -2.5322515 -1.035571   2.5982357]: error rate = 0.15
exp3 - wp [ 1.         -2.262239   -2.58554171 -1.625246    2.871206  ]: error rate = 0.102
exp4 - wp [ 1.        -1.37192   -2.3714986 -1.374279   1.9456777]: error rate = 0.1
exp5 - wp [ 2.        -2.401157  -3.0646863 -1.866475   1.725899 ]: error rate = 0.132
exp6 - wp [ 1.        -2.653819  -1.886842  -1.3327683  2.6465114]: error rate = 0.126
exp7 - wp [ 2.       -1.979648 -2.209483 -1.784171  1.301244]: error rate = 0.158
exp8 - wp [ 2.         -1.913025   -2.5540523  -1.80439     1.38198738]: error rate = 0.148
exp9 - wp [ 1.        -2.325228  -2.4428204 -1.099303   2.3118668]: error rate = 0.104
exp10 - wp [ 1.         -2.077392   -2.78586061 -1.347896    2.743228  ]: error rate = 0.104
exp11 - wp [ 1.        -1.792569  -1.

exp94 - wp [ 1.         -2.480034   -2.494324   -1.381645    3.25270438]: error rate = 0.118
exp95 - wp [ 1.       -1.672094 -2.045499 -1.274204  2.117085]: error rate = 0.1
exp96 - wp [ 1.         -1.338946   -2.44775181 -1.489808    2.25376   ]: error rate = 0.11
exp97 - wp [ 0.         -1.047131   -1.16310561 -0.827747    1.92208   ]: error rate = 0.134
exp98 - wp [ 2.         -2.17569    -2.73320261 -2.110384    1.94581   ]: error rate = 0.128
exp99 - wp [ 1.         -1.790175   -2.69553351 -1.323882    2.64026978]: error rate = 0.106
exp100 - wp [ 2.       -2.838282 -2.786886 -2.092528  2.713275]: error rate = 0.122
exp101 - wp [ 1.       -1.714666 -1.637573 -1.5983    1.930837]: error rate = 0.102
exp102 - wp [ 0.        -1.829501  -2.031439  -0.6797655  2.755547 ]: error rate = 0.146
exp103 - wp [ 1.         -1.323148   -2.04451161 -1.337398    1.721341  ]: error rate = 0.1
exp104 - wp [ 1.         -1.480517   -2.7338675  -1.4488877   2.32648338]: error rate = 0.106
exp105 - wp 

exp188 - wp [ 0.        -1.051709  -1.4663592 -1.085588   2.212859 ]: error rate = 0.146
exp189 - wp [ 1.        -1.651529  -2.5821201 -1.267026   2.272493 ]: error rate = 0.1
exp190 - wp [ 1.       -2.131857 -2.197681 -1.419576  2.468972]: error rate = 0.1
exp191 - wp [ 1.       -1.455317 -1.64024  -1.190438  1.382241]: error rate = 0.1
exp192 - wp [ 1.         -1.561574   -2.85491511 -1.401398    2.4502757 ]: error rate = 0.106
exp193 - wp [ 1.         -1.551836   -2.49586281 -1.700293    2.4989811 ]: error rate = 0.102
exp194 - wp [ 2.        -2.654754  -3.0571756 -1.343984   1.9237557]: error rate = 0.122
exp195 - wp [ 2.        -2.774177  -2.703107  -2.1868245  2.11     ]: error rate = 0.116
exp196 - wp [ 2.       -1.822512 -3.608666 -1.781321  2.141475]: error rate = 0.156
exp197 - wp [ 1.        -2.768338  -3.2394585 -1.8659337  3.9253507]: error rate = 0.108
exp198 - wp [ 1.       -2.256304 -2.245902 -1.1574    2.376409]: error rate = 0.1
exp199 - wp [ 1.         -1.850549   -2

exp281 - wp [ 1.        -1.786448  -2.4209816 -1.216739   1.8252817]: error rate = 0.116
exp282 - wp [ 1.        -1.583269  -2.789333  -1.3927896  2.48424  ]: error rate = 0.108
exp283 - wp [ 1.         -2.21204    -2.62590911 -1.328645    2.565359  ]: error rate = 0.104
exp284 - wp [ 1.       -1.426221 -1.855791 -1.567303  2.187415]: error rate = 0.122
exp285 - wp [ 1.       -1.695065 -1.605952 -0.885667  1.333577]: error rate = 0.108
exp286 - wp [ 1.        -1.492702  -2.2025425 -1.10034    1.7742013]: error rate = 0.1
exp287 - wp [ 2.        -2.711893  -2.9324712 -1.8046213  2.034957 ]: error rate = 0.11
exp288 - wp [ 2.        -2.846683  -3.9641854 -0.950847   2.232168 ]: error rate = 0.128
exp289 - wp [ 1.        -2.342688  -2.4807812 -1.607819   2.7365831]: error rate = 0.102
exp290 - wp [ 1.        -2.61164   -2.3505615 -0.730932   2.738892 ]: error rate = 0.134
exp291 - wp [ 1.       -2.306608 -1.735546 -0.926898  2.075314]: error rate = 0.126
exp292 - wp [ 1.       -2.552893 -

exp375 - wp [ 1.         -1.865662   -2.29379061 -1.82011     2.519653  ]: error rate = 0.1
exp376 - wp [ 2.         -1.683225   -3.74952891 -1.799678    2.009295  ]: error rate = 0.164
exp377 - wp [ 1.        -2.357052  -2.0834945 -1.609527   2.6273027]: error rate = 0.102
exp378 - wp [ 2.        -2.084146  -3.1975139 -2.287181   2.303091 ]: error rate = 0.126
exp379 - wp [ 1.        -1.514602  -2.5732141 -1.557996   2.40623  ]: error rate = 0.102
exp380 - wp [ 1.         -2.311993   -2.30876861 -2.2423995   3.071158  ]: error rate = 0.104
exp381 - wp [ 1.       -2.127123 -2.443863 -1.156312  2.155013]: error rate = 0.102
exp382 - wp [ 2.         -2.032808   -2.8628478  -1.704816    1.80654008]: error rate = 0.148
exp383 - wp [ 2.         -2.395771   -3.65083561 -2.247725    2.80706   ]: error rate = 0.11
exp384 - wp [ 1.       -1.238235 -1.415929 -0.976628  1.12582 ]: error rate = 0.116
exp385 - wp [ 1.         -2.311366   -2.25583571 -1.06395     2.755353  ]: error rate = 0.12
exp38

...

exp1582 - wp [ 1.        -1.793639  -2.5875323 -1.549612   2.44451  ]: error rate = 0.1
exp1583 - wp [ 1.         -2.105092   -1.98152061 -0.791791    1.93122   ]: error rate = 0.12
exp1584 - wp [ 1.         -2.151677   -2.01591723 -1.671017    2.320635  ]: error rate = 0.102
exp1585 - wp [ 1.       -1.195633 -2.417382 -1.681938  2.049586]: error rate = 0.126
exp1586 - wp [ 1.       -3.080839 -2.012114 -2.116565  3.511726]: error rate = 0.126
exp1587 - wp [ 1.         -1.617463   -1.73742291 -1.562937    2.00803   ]: error rate = 0.104
exp1588 - wp [ 2.       -4.097998 -2.602234 -2.115969  3.138831]: error rate = 0.134
exp1589 - wp [ 1.        -2.036013  -1.8729413 -1.379956   2.214397 ]: error rate = 0.106
exp1590 - wp [ 1.         -1.712218   -2.37989281 -1.6478246   2.522476  ]: error rate = 0.1
exp1591 - wp [ 1.        -2.826621  -2.5776493 -1.632656   3.482796 ]: error rate = 0.112
exp1592 - wp [ 1.        -2.099888  -2.2035704 -1.3521207  2.4675517]: error rate = 0.1
exp1593 - wp

exp1674 - wp [ 2.         -2.599214   -3.42782184 -1.491065    2.3344651 ]: error rate = 0.118
exp1675 - wp [ 0.       -1.18739  -0.693265 -0.50561   1.61122 ]: error rate = 0.158
exp1676 - wp [ 2.         -2.770477   -3.77991643 -1.61511     2.5347394 ]: error rate = 0.114
exp1677 - wp [ 1.       -1.672201 -2.548442 -1.122868  2.175081]: error rate = 0.102
exp1678 - wp [ 1.        -0.92046   -1.636767  -1.6439027  1.384865 ]: error rate = 0.14
exp1679 - wp [ 1.        -2.654413  -3.3252302 -2.087263   3.532432 ]: error rate = 0.118
exp1680 - wp [ 1.         -2.034727   -2.5247498  -1.451973    2.35836408]: error rate = 0.1
exp1681 - wp [ 1.        -1.239784  -2.028922  -1.4698975  1.607305 ]: error rate = 0.114
exp1682 - wp [ 1.         -2.056129   -2.19933961 -1.031542    2.008993  ]: error rate = 0.102
exp1683 - wp [ 2.        -2.06744   -2.8176257 -2.310491   1.9473611]: error rate = 0.132
exp1684 - wp [ 1.       -2.479664 -2.188283 -1.742654  2.937665]: error rate = 0.11
exp1685 -

exp1768 - wp [ 1.       -1.782859 -2.001603 -1.582237  2.04273 ]: error rate = 0.102
exp1769 - wp [ 1.         -2.171635   -2.03285351 -1.605856    2.665263  ]: error rate = 0.108
exp1770 - wp [ 2.       -3.402839 -3.355686 -2.164889  3.229946]: error rate = 0.102
exp1771 - wp [ 2.        -1.521118  -3.0299484 -2.6591886  1.9712783]: error rate = 0.16
exp1772 - wp [ 1.        -2.518392  -2.523339  -1.5034847  3.22755  ]: error rate = 0.11
exp1773 - wp [ 1.        -1.92261   -1.9436855 -0.91674    1.8266467]: error rate = 0.106
exp1774 - wp [ 1.        -2.796302  -2.1524255 -1.485689   2.8311237]: error rate = 0.122
exp1775 - wp [ 2.        -2.194007  -2.7906415 -1.898173   1.8797963]: error rate = 0.128
exp1776 - wp [ 1.         -2.273998   -2.02893251 -1.1666      2.413907  ]: error rate = 0.116
exp1777 - wp [ 1.         -2.739118   -2.03347031 -1.971257    3.0641507 ]: error rate = 0.122
exp1778 - wp [ 2.        -2.329293  -3.393592  -1.6243736  2.061853 ]: error rate = 0.124
exp1779

exp1860 - wp [ 1.        -2.071994  -3.3453825 -1.633657   3.1037787]: error rate = 0.108
exp1861 - wp [ 1.        -2.329067  -2.696066  -0.687145   2.7137053]: error rate = 0.132
exp1862 - wp [ 0.       -1.465706 -2.331766 -2.277008  4.03667 ]: error rate = 0.144
exp1863 - wp [ 1.         -2.381506   -1.84914261 -2.587355    3.000854  ]: error rate = 0.122
exp1864 - wp [ 1.         -2.836098   -2.54057161 -1.677031    3.25550738]: error rate = 0.108
exp1865 - wp [ 1.        -1.432619  -2.5411305 -1.797401   2.488063 ]: error rate = 0.108
exp1866 - wp [ 1.         -2.746838   -2.29866881 -1.794998    3.370659  ]: error rate = 0.118
exp1867 - wp [ 1.       -2.167193 -2.542342 -1.521194  2.87356 ]: error rate = 0.1
exp1868 - wp [ 1.        -1.899041  -3.2948072 -1.217976   3.0598194]: error rate = 0.118
exp1869 - wp [ 1.       -2.314422 -2.268915 -1.375461  3.050364]: error rate = 0.12
exp1870 - wp [ 1.       -1.878543 -2.066165 -0.999862  1.847098]: error rate = 0.102
exp1871 - wp [ 1. 

exp1952 - wp [ 2.       -3.197461 -3.740213 -2.528771  3.724398]: error rate = 0.1
exp1953 - wp [ 1.        -2.487733  -2.8926042 -1.0584847  3.147955 ]: error rate = 0.118
exp1954 - wp [ 1.        -1.315758  -2.4175389 -1.613413   2.2824247]: error rate = 0.11
exp1955 - wp [ 1.        -1.649988  -2.0901675 -1.507724   2.0338923]: error rate = 0.1
exp1956 - wp [ 1.        -2.233276  -3.4282238 -1.442619   3.1458077]: error rate = 0.11
exp1957 - wp [ 2.       -1.835618 -3.640721 -2.195962  2.174697]: error rate = 0.148
exp1958 - wp [ 1.        -2.421582  -2.7170268 -1.5411356  3.2888277]: error rate = 0.104
exp1959 - wp [ 1.        -2.304727  -2.8710429 -1.776296   3.1227   ]: error rate = 0.106
exp1960 - wp [ 2.        -2.532241  -3.9125655 -1.634032   2.6498847]: error rate = 0.122
exp1961 - wp [ 1.       -2.0823   -2.535238 -1.961836  2.714188]: error rate = 0.112
exp1962 - wp [ 2.        -2.591411  -3.5555828 -1.544638   2.3452053]: error rate = 0.116
exp1963 - wp [ 1.        -2.281